In [137]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report

In [3]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    print(result)
    return result

In [4]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [5]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("./data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

[-5.37004395e+02  3.16664009e+01 -6.54209137e+00  4.56760550e+00
 -7.38780499e+00 -1.36847029e+01 -1.78605824e+01 -1.18284941e+01
 -2.25547522e-01 -1.18684692e+01 -8.85833073e+00  2.02371097e+00
 -9.43830681e+00 -4.10037184e+00 -6.45495605e+00 -3.87169814e+00
 -4.01648903e+00 -4.76854229e+00 -5.11756361e-01  1.33889771e+00
  2.94026041e+00  3.80391908e+00  2.59524417e+00  5.09492159e+00
 -7.14929879e-01  2.14967346e+00 -1.82484186e+00  7.07248390e-01
 -4.69545782e-01 -3.38976714e-03 -5.91821849e-01 -1.61437988e-02
  1.01429634e-01  8.82703602e-01  2.87238210e-01  3.08487684e-01
 -8.90635967e-01 -1.47073328e+00 -7.07513273e-01 -3.91681731e-01
  4.82439458e-01  4.45954621e-01  4.22002167e-01  3.90913785e-01
  3.83992523e-01  4.37711865e-01  4.52970654e-01  4.81909484e-01
  5.30166030e-01  5.87507665e-01  5.77323794e-01  5.37041843e-01
  2.69676730e-05  3.71418246e-05  4.45121150e-05  6.87977517e-05
  7.41148615e-05  1.02672179e-03  8.21552984e-03  5.98948188e-02
  1.94896534e-01  2.97292

In [7]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [25]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


## 1. Multilayer perceptron

In [117]:
def MLP(x_train, y_train):
    # define the multinomial logistic regression model
    model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
    #DataFlair - Train the model
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [118]:
scores = MLP(x_train, y_train)

In [119]:
scores

array([0.39655172, 0.37931034, 0.51724138, 0.51724138, 0.46551724,
       0.55172414, 0.52631579, 0.45614035, 0.66666667, 0.64912281,
       0.43103448, 0.4137931 , 0.44827586, 0.48275862, 0.53448276,
       0.4137931 , 0.61403509, 0.54385965, 0.40350877, 0.52631579,
       0.44827586, 0.37931034, 0.43103448, 0.55172414, 0.5       ,
       0.65517241, 0.49122807, 0.61403509, 0.54385965, 0.61403509])

In [120]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.506 (0.083)


In [160]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [161]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [162]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [163]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 61.98%


In [164]:
f1=f1_score(y_true=y_test, y_pred=y_pred, average="macro")
f1

0.6087919536462651

In [165]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        calm       0.86      0.53      0.66        47
     disgust       0.51      0.85      0.64        55
     fearful       0.71      0.39      0.51        38
       happy       0.65      0.62      0.63        52

    accuracy                           0.62       192
   macro avg       0.68      0.60      0.61       192
weighted avg       0.67      0.62      0.62       192



## 2. Logistic regression

In [141]:
def logistic_regression(x_train, y_train):
    # define the multinomial logistic regression model
    model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=500,  penalty='l2')
    #DataFlair - Train the model
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [142]:
scores = logistic_regression(x_train, y_train)

In [143]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.636 (0.061)


In [144]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model = LogisticRegression(multi_class='multinomial', solver='newton-cg', max_iter=100,  penalty='l2')

In [145]:
#DataFlair - Train the model
model.fit(x_train,y_train)

LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [146]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [147]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 64.58%


In [148]:
f1=f1_score(y_true=y_test, y_pred=y_pred, average="macro")
f1

0.6421808081771366

In [149]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        calm       0.69      0.77      0.73        47
     disgust       0.67      0.64      0.65        55
     fearful       0.55      0.61      0.57        38
       happy       0.65      0.58      0.61        52

    accuracy                           0.65       192
   macro avg       0.64      0.65      0.64       192
weighted avg       0.65      0.65      0.65       192



## 3. SVM

In [150]:
def SVM(x_train, y_train):
    # define the multinomial logistic regression model
    model = svm.SVC(kernel='poly', gamma=0.5, C=0.1)
    #DataFlair - Train the model
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model and collect the scores
    n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

    return n_scores

In [151]:
scores = SVM(x_train, y_train)

In [153]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.665 (0.057)


In [154]:
scores

array([0.56896552, 0.65517241, 0.68965517, 0.67241379, 0.60344828,
       0.74137931, 0.64912281, 0.64912281, 0.61403509, 0.61403509,
       0.70689655, 0.68965517, 0.63793103, 0.67241379, 0.74137931,
       0.81034483, 0.66666667, 0.71929825, 0.59649123, 0.68421053,
       0.5862069 , 0.65517241, 0.60344828, 0.68965517, 0.67241379,
       0.72413793, 0.57894737, 0.75438596, 0.59649123, 0.70175439])

In [155]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model = svm.SVC(kernel='poly', gamma=0.5, C=0.1)

In [156]:
model.fit(x_train,y_train)

SVC(C=0.1, gamma=0.5, kernel='poly')

In [157]:
y_pred=model.predict(x_test)

In [158]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%


In [159]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        calm       0.73      0.74      0.74        47
     disgust       0.75      0.71      0.73        55
     fearful       0.64      0.66      0.65        38
       happy       0.74      0.75      0.74        52

    accuracy                           0.72       192
   macro avg       0.71      0.72      0.71       192
weighted avg       0.72      0.72      0.72       192

